# Genetic optimisation of SVC classifier for the SDSS data

## Load data

In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals
import warnings
import helpers
from helpers import DataSet
import matplotlib as mpl
import os
import random

# Common imports
import pandas as pd
import numpy as np
import seaborn as sns

# Imports for ML
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier, PassiveAggressiveClassifier, RidgeClassifier, Perceptron
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import RobustScaler, StandardScaler, MaxAbsScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.decomposition import PCA

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
sns.set_style('whitegrid')

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "results"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)

# Helper functioins and structures
# Ignore useless warnings (see SciPy issue #5998)
warnings.filterwarnings(action="ignore", message="^internal gelsd")

DATA_PATH = "Skyserver_SQL2_27_2018 6_51_39 PM.csv"
RESULTS_FOLDER = "results"

# We load the data. Those that have nothing to do with the features of the objects are ignored.
sdss_data = pd.read_csv(DATA_PATH)

# We have a general look at the features
sdss_data.head(3)

print(sdss_data.columns.values)
sdss_data.drop(['objid', 'run', 'rerun', 'camcol', 'field',
                'specobjid', 'fiberid', 'mjd', 'plate'], axis=1, inplace=True)
sdss_data.head(1)

sdss_df_fe = sdss_data

# Principal Component Analysis
pca = PCA(n_components=3)
ugriz = pca.fit_transform(sdss_df_fe[['u', 'g', 'r', 'i', 'z']])

# update dataframe 
sdss_df_fe = pd.concat((sdss_df_fe, pd.DataFrame(ugriz)), axis=1)
sdss_df_fe.rename({0: 'PCA_1', 1: 'PCA_2', 2: 'PCA_3'}, axis=1, inplace = True)
sdss_df_fe.drop(['u', 'g', 'r', 'i', 'z'], axis=1, inplace=True)
sdss_df_fe.head()

X = sdss_data.drop(['class'], axis=1)
y = sdss_data['class']

class_names = ["GALAXY", "QSO", "STAR"]

std_scaler = StandardScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

X_test = std_scaler.fit_transform(X_test.astype(np.float64))
X_train = std_scaler.fit_transform(X_train.astype(np.float64))

['objid' 'ra' 'dec' 'u' 'g' 'r' 'i' 'z' 'run' 'rerun' 'camcol' 'field'
 'specobjid' 'class' 'redshift' 'plate' 'mjd' 'fiberid']


## Optimize

In [2]:
from evolutionary_search import EvolutionaryAlgorithmSearchCV

paramgrid = {"base_estimator": [Perceptron(), DecisionTreeClassifier()],
             "n_estimators": np.logspace(2, 4, base=10, num=25, dtype=int),
             "max_samples": np.logspace(0, 2, base=10, num=25, dtype=int),
             "max_features": np.logspace(0, 2, base=10, num=25, dtype=int),
             "bootstrap": [True, False],
             "bootstrap_features": [False, True],
             "oob_score": [False, True],
             "n_jobs": [4],
             "random_state": [42],
             }

random.seed(1)

cv = EvolutionaryAlgorithmSearchCV(estimator=BaggingClassifier(),
                                   params=paramgrid,
                                   scoring="accuracy",
                                   cv=StratifiedKFold(n_splits=5),
                                   verbose=3,
                                   population_size=50,
                                   gene_mutation_prob=0.10,
                                   gene_crossover_prob=0.5,
                                   tournament_size=3,
                                   generations_number=15,
                                   n_jobs=8,
                                   error_score=0.0)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    result = cv.fit(X, y)

Types [1, 1, 1, 1, 1, 1, 1, 1, 1] and maxint [1, 24, 24, 24, 1, 1, 1, 0, 0] detected
[CV] base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), n_estimators=562, max_samples=31, max_features=17, bootstrap=False, bootstrap_features=False, oob_score=True, n_jobs=4, random_state=42 
[CV] base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), n_estimators=8254, max_samples=3, ma

      validation_fraction=0.1, verbose=0, warm_start=False), n_estimators=825, max_samples=3, max_features=56, bootstrap=True, bootstrap_features=True, oob_score=True, n_jobs=4, random_state=42, score=0.0, total=   0.0s[CV]  base_estimator=Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
      fit_intercept=True, max_iter=None, n_iter=None, n_iter_no_change=5,
      n_jobs=None, penalty=None, random_state=0, shuffle=True, tol=None,
      validation_fraction=0.1, verbose=0, warm_start=False), n_estimators=3162, max_samples=100, max_features=1, bootstrap=False, bootstrap_features=False, oob_score=True, n_jobs=4, random_state=42, score=0.0, total=   0.0s

[CV] base_estimator=Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
      fit_intercept=True, max_iter=None, n_iter=None, n_iter_no_change=5,
      n_jobs=None, penalty=None, random_state=0, shuffle=True, tol=None,
      validation_fraction=0.1, verbose=0, warm_start=False), n_estima

      validation_fraction=0.1, verbose=0, warm_start=False), n_estimators=261, max_samples=46, max_features=82, bootstrap=False, bootstrap_features=False, oob_score=True, n_jobs=4, random_state=42, score=0.0, total=   0.0s
[CV]  base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), n_estimators=825, max_samples=17, max_features=82, bootstrap=True, bootstrap_features=True, oob_score=False, n_jobs=4, random_state=42, score=0.0, total=   0.0s

[CV] base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1,

            splitter='best'), n_estimators=5623, max_samples=2, max_features=8, bootstrap=False, bootstrap_features=False, oob_score=True, n_jobs=4, random_state=42 
[CV]  base_estimator=Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
      fit_intercept=True, max_iter=None, n_iter=None, n_iter_no_change=5,
      n_jobs=None, penalty=None, random_state=0, shuffle=True, tol=None,
      validation_fraction=0.1, verbose=0, warm_start=False), n_estimators=1778, max_samples=8, max_features=31, bootstrap=True, bootstrap_features=True, oob_score=True, n_jobs=4, random_state=42, score=0.0, total=   0.0s
[CV] base_estimator=Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
      fit_intercept=True, max_iter=None, n_iter=None, n_iter_no_change=5,
      n_jobs=None, penalty=None, random_state=0, shuffle=True, tol=None,
      validation_fraction=0.1, verbose=0, warm_start=False), n_estimators=1778, max_samples=8, max_features=31, bootstrap=True

            splitter='best'), n_estimators=215, max_samples=1, max_features=82, bootstrap=True, bootstrap_features=True, oob_score=True, n_jobs=4, random_state=42 

[CV]  base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), n_estimators=316, max_samples=1, max_features=17, bootstrap=True, bootstrap_features=True, oob_score=True, n_jobs=4, random_state=42, score=0.0, total=   0.0s[CV]  base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.

...

            splitter='best'), n_estimators=261, max_samples=46, max_features=6, bootstrap=False, bootstrap_features=False, oob_score=False, n_jobs=4, random_state=42 
[CV]  base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), n_estimators=261, max_samples=46, max_features=6, bootstrap=False, bootstrap_features=False, oob_score=False, n_jobs=4, random_state=42, score=0.9784892446223111, total=   0.5s
[CV] base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_wei

            splitter='best'), n_estimators=261, max_samples=46, max_features=6, bootstrap=False, bootstrap_features=False, oob_score=False, n_jobs=4, random_state=42 
[CV]  base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), n_estimators=562, max_samples=46, max_features=2, bootstrap=False, bootstrap_features=False, oob_score=False, n_jobs=4, random_state=42, score=0.9409704852426213, total=   1.0s
[CV]  base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_we

            splitter='best'), n_estimators=261, max_samples=46, max_features=6, bootstrap=False, bootstrap_features=False, oob_score=False, n_jobs=4, random_state=42, score=0.9784892446223111, total=   0.7s
[CV]  base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), n_estimators=261, max_samples=46, max_features=6, bootstrap=False, bootstrap_features=False, oob_score=False, n_jobs=4, random_state=42, score=0.9795102448775612, total=   0.6s
[CV] base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, 

            splitter='best'), n_estimators=261, max_samples=46, max_features=6, bootstrap=False, bootstrap_features=False, oob_score=False, n_jobs=4, random_state=42 
[CV]  base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), n_estimators=261, max_samples=46, max_features=6, bootstrap=False, bootstrap_features=False, oob_score=False, n_jobs=4, random_state=42, score=0.9814907453726863, total=   0.5s
[CV] base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_wei

            splitter='best'), n_estimators=261, max_samples=46, max_features=6, bootstrap=False, bootstrap_features=False, oob_score=False, n_jobs=4, random_state=42, score=0.9784892446223111, total=   0.5s
[CV]  base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), n_estimators=261, max_samples=46, max_features=6, bootstrap=False, bootstrap_features=False, oob_score=False, n_jobs=4, random_state=42, score=0.9814907453726863, total=   0.5s
[CV] base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, 

In [3]:
best_indiv = LinearSVC(C = 5623.413251903491, 
                     penalty = 'l1',
                     loss = 'squared_hinge',
                     dual = False,
                     multi_class = 'ovr',
                     fit_intercept = True,
                     intercept_scaling = 0.03162277660168379,
                     random_state = 42,
                     tol = 1e-05,
                     max_iter = 316.22776601683796)